In [78]:
import config
import pymongo
import numpy as np
import pandas as pd
import json
import re
import simplejson
from math import isnan
import ast
#import attributes

import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection
from sqlalchemy.types import VARCHAR

from pandas_profiling import ProfileReport
from ipywidgets import interact, Dropdown

In [81]:
myclient = pymongo.MongoClient(config.MONGO_URI)
stagingDb = myclient[config.DB_STAGING]
analyseDb = myclient[config.DB_ANALYSE]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]
metaCollection = stagingDb['Kolominformatie']

AIRFLOW_WASSTRAAT_CONFIG = "./wasstraat_config/Wasstraat_Config_Harmonize.xlsx"
AIRFLOW_WASSTRAAT_CONFIG_NEW = "./wasstraat_config/Wasstraat_Config_Harmonize_new.xlsx"

SUGGESTIE_XL = "./wasstraat_config/Wasstraat_Attribuut_Suggestie_v2.xlsx"

In [82]:
show_fieldsets = [
        ("Projectvelden", {"fields": ["project", "put", "vlaknr", "spoor", "vondstnr"]}),
        ("inhoudvelden", {"fields": ["inhoud", "omstandigheden", "segment", "vaknummer", "verzamelwijze"]}),
        ("Datering", {"fields": ["dateringvanaf", "dateringtot", "datering"]})]
show_fieldsets

[('Projectvelden',
  {'fields': ['project', 'put', 'vlaknr', 'spoor', 'vondstnr']}),
 ('inhoudvelden',
  {'fields': ['inhoud',
    'omstandigheden',
    'segment',
    'vaknummer',
    'verzamelwijze']}),
 ('Datering', {'fields': ['dateringvanaf', 'dateringtot', 'datering']})]

In [83]:
show_fieldsets_columns = [
        ("Projectvelden", {"columns": [
            {"fields": ["project", "put", "vlaknr", "spoor", "vondstnr", "discriminator"], 'grid':4},
            {"fields": ["project", "put", "vlaknr", "spoor", "vondstnr"], 'grid':3},
            {"fields": ["project", "put", "vlaknr", "spoor", "vondstnr"], 'gridt':3},

        ]}),
        ("inhoudvelden", {"fields": ["inhoud", "omstandigheden", "segment", "discriminator", "verzamelwijze"]}),
        ("Datering", {"fields": ["dateringvanaf", "dateringtot", "datering"]})]

show_fieldsets_columns[0][1]['columns']

[{'fields': ['project', 'put', 'vlaknr', 'spoor', 'vondstnr', 'discriminator'],
  'grid': 4},
 {'fields': ['project', 'put', 'vlaknr', 'spoor', 'vondstnr'], 'grid': 3},
 {'fields': ['project', 'put', 'vlaknr', 'spoor', 'vondstnr'], 'gridt': 3}]

In [94]:
df_artefact = pd.DataFrame(list(analyseColClean.find({'soort': 'Artefact'})))
lst_artefactcols = df_artefact.columns

df_artefact.dtypes

_id                       object
brondata                  object
projectcd                 object
soort                     object
artefactsoort             object
                           ...  
slijtage_onderkaaks_M1    object
slijtage_onderkaaks_M2    object
slijtage_onderkaaks_M3    object
symmetrie                 object
vergroeiing               object
Length: 193, dtype: object

In [93]:
#df_artefact.iloc[0]['soort'][:80]
#df_artefact = df_artefact.astype({"soort": str, "artefactsoort": str})
df_artefact['soort'] = df_artefact['soort'].apply(str)

AttributeError: 'VARCHAR(80)' is not a valid function for 'Series' object

In [92]:
lst_tables = ['Def_Project', 'Def_Put', 'Def_Vondst', 'Def_Spoor', 'Def_Stelling', 'Def_Doos', 'Def_Standplaats', 'Def_Plaatsing', 'Def_Vlak', 'Def_Vindplaats', 'Def_Artefact'] #['Def_Project', 'Def_Vondst', 'Def_Stelling', 'Def_Plaatsing', 'Def_Vindplaats', 'Def_Artefact']
regexProject = re.compile(r'\'project\': \'(.*?)\'') # regex to replace Object




engine = create_engine(config.SQLALCHEMY_DATABASE_URI)
with engine.connect() as connection:
    insp = inspect(connection)
    db_columns = insp.get_columns('Def_Artefact')
    
    lst_columns = [col['name'] for col in db_columns] 
    lst_doorsnede = [col for col in lst_columns if col in lst_artefactcols]
    
    for col in lst_doorsnede:
        for dbcol in [dbcol for dbcol in db_columns if dbcol['name'] == col]:
            if type(dbcol['type']) == VARCHAR and col != '_id':
                length = dbcol["type"].length
                print(f'Column {col} with length {length}')
                df_artefact[col] = df_artefact[col].apply(str)
                #df_artefact = df_artefact.astype({"soort": str, "artefactsoort": str})
                #df_artefact[col] = df_artefact.apply(lambda x: x[col][:dbcol['type'].length] if type(x[col]) == str else '', axis=1)
                df_artefact[col] = df_artefact[col][:length]
            pass
                
                
    
db_columns[0]
df_artefact.head(5)

Column soort with length 80


AttributeError: 'VARCHAR(80)' is not a valid function for 'Series' object

In [62]:
type(df_artefact.iloc[0]['soort'])

str